In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import rotate
import Utils
from Utils import Constants
import cv2
from facenet_pytorch import InceptionResnetV1
from Models import *
from DataLoaders import *

In [3]:
train_labels = pd.read_csv('train_data_clean.csv')
test_labels = pd.read_csv('test_data_clean.csv')
validation_labels = pd.read_csv('validation_data_clean.csv')
train_labels

,name,skin_tone,gender,age,is_face
0,TRAIN0001.png,0,0,1,False
1,TRAIN0002.png,5,1,0,True
2,TRAIN0005.png,1,1,0,False
3,TRAIN0007.png,1,0,1,True
4,TRAIN0009.png,7,0,1,False
...,...,...,...,...,...
6837,TRAIN9992.png,4,0,2,True
6838,TRAIN9993.png,1,1,1,True
6839,TRAIN9995.png,8,0,1,True
6840,TRAIN9998.png,4,1,1,False


In [4]:
def categorical_accuracy(ypred,y):
    #y is index, ypred i s one hot like in loss functions
    predicted = torch.argmax(ypred,1).long()
    correct = torch.mean((y.long() == predicted).float())
    return correct

def save_train_history(model,history,root=''):
    model_name = model.get_identifier()
    
    df = pd.DataFrame(history)
    df['model'] = model_name
    string = root + 'results/history_' + model_name + '.csv'
    df.to_csv(string,index=False)
    return df, string

def train_model(model,
                train_df,
                validation_df,
                root,
                epochs=300,
                lr=.01,
                batch_size=200,
                patience = 20,
                loss_weights = [2,1,.5],
                save_path=None,
                histogram = True,
                upsample=True,
                **kwargs,
               ):
    if save_path is None:
        save_path = root + 'models/'+ model.get_identifier()
        if upsample:
            save_path += '_balanced'
    if upsample:
        patience = int(patience/10) + 1
    train_loader = FaceGenerator(train_df,Constants.data_root,batch_size=batch_size,upsample=upsample,**kwargs)
    validation_loader = FaceGenerator(validation_df,Constants.data_root,validation=True,batch_size=batch_size,upsample=upsample,**kwargs)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train(True)
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    format_y = lambda y: y.long().to(device)
    
    def get_loss(m,xin,ytrue):
        outputs = m(xin)
        losses = [loss_fn(ypred.float(),format_y(y)) for y,ypred in zip(ytrue,outputs)]
        l1 = torch.mul(loss_weights[0],losses[0])
        l2 =  torch.mul(loss_weights[1],losses[1])
        l3 =  torch.mul(loss_weights[2],losses[2])
        total_losses = l1 + l2 + l3
        return outputs,total_losses
    
    def train_epoch():
        running_loss = 0
        running_accuracy = [0,0,0]
        curr_loss = 0
        count = 0
        for i, [x_batch, y_batch] in enumerate(train_loader):
            optimizer.zero_grad()
            if histogram:
                xh = torch_color_histogram(x_batch)
                xh = xh.to(device)
                xh.requires_grad = True
                xxb = x_batch.to(device)
                xxb.requires_grad=True
                xb = [xxb,xh]
            else:
                xb = x_batch.to(device)
                xb.requires_grad_(True)
            outputs,total_losses = get_loss(model, xb,y_batch)
            total_losses.backward()
            optimizer.step()
            running_loss += total_losses.item()
            print('curr loss',total_losses.item(), 'step',i,' | ',end='\r')
            count += 1
            with torch.no_grad():
                for i,(y,ypred) in enumerate(zip(y_batch,outputs)):
                    accuracy = categorical_accuracy(ypred.float(),format_y(y))
                    running_accuracy[i] += accuracy.item()
        return running_loss/count, [a/count for a in running_accuracy]
    
    def val_epoch():
        running_loss = 0
        running_accuracy = [0,0,0]
        count = 0
        for i, [x_batch, y_batch] in enumerate(validation_loader):
            if histogram:
                xh = torch_color_histogram(x_batch)
                xh = xh.to(device)
                xxb = x_batch.to(device)
                xb = [xxb,xh]
            else:
                xb = x_batch.to(device)
            outputs = model(xb)
            outputs,total_losses = get_loss(model,xb,y_batch)
            running_loss += total_losses.item()
            count += 1
            for i,(y,ypred) in enumerate(zip(y_batch, outputs)):
                accuracy = categorical_accuracy(ypred.float(),format_y(y))
                running_accuracy[i] += accuracy.item()
        return running_loss/count, [a/count for a in running_accuracy]
    
    
    best_val_loss = 100000
    steps_since_improvement = 0
    hist = []
    best_weights = model.state_dict()
    print('model being saved to',save_path)
    for epoch in range(epochs):
        print('epoch',epoch)
        model.train(True)
        avg_loss, avg_acc = train_epoch()
        print('train loss', avg_loss, 'train accuracy', avg_acc)
        model.train(False)
        val_loss, val_acc = val_epoch()
        print('val loss', val_loss, 'val accuracy', val_acc)
#         torch.save(model.state_dict(), save_path + '_epoch' + str(epoch))
        if best_val_loss > val_loss:
            torch.save(model,save_path)
            best_weights = model.state_dict()
            best_val_loss = val_loss
            steps_since_improvement = 0
        else:
            steps_since_improvement += 1
        
        hist_entry = {
            'epoch': epoch,
            'train_loss': avg_loss,
            'train_acc':avg_acc,
            'val_loss':val_loss,
            'val_acc': val_acc,
            'lr': lr,
            'loss_weights': '_'.join([str(l) for l in loss_weights])
        }
        hist.append(hist_entry)
        save_train_history(model,hist,root=root)
        if steps_since_improvement > patience:
            break
    return model,hist

m,h = train_model(
    DualFacenetModel(embedding_dropout=.4,hidden_dims=[600]),
    train_labels,
    validation_labels,
    Constants.data_root,
    batch_size=100,
    histogram=False,
#     lr=10,
#     upsample=False,
)
del m
h

(35354, 5)
(7786, 5)
model being saved to ../../data/models/dual_dualfacenet_h600_st600_a400_g400_ed4_std2_ad2_gd2_balanced
epoch 0


/opt/conda/envs/rapids/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train loss 6.122271144457456 train accuracy [0.17483782764040146, 0.4327495172198883, 0.6112010759317269]
val loss 5.852380599731054 val accuracy [0.2356529458402059, 0.5615623031671231, 0.7665086289246877]
epoch 1
train loss 6.080811311969649 train accuracy [0.19652018750791495, 0.45214269575426136, 0.6336796269410074]
val loss 5.938020700063461 val accuracy [0.18563207640097693, 0.5885688639604129, 0.7551162609687219]
epoch 2
train loss 6.119507001618207 train accuracy [0.18565285127967765, 0.4603075832970398, 0.641798479745617]
val loss 6.0662884162022515 val accuracy [0.15022957277221557, 0.5447853173200901, 0.757576013986881]
epoch 3
train loss 6.167756818782138 train accuracy [0.1758589619339186, 0.45036826724723233, 0.6375695606746242]
val loss 6.0802591397212105 val accuracy [0.13117173132606041, 0.5702325445719254, 0.7920393370665036]
epoch 4
train loss 6.252699257963795 train accuracy [0.15820673399599594, 0.4140165198152348, 0.6331230282446759]
val loss 6.341916750638913 val

[{'epoch': 0,
  'train_loss': 6.122271144457456,
  'train_acc': [0.17483782764040146, 0.4327495172198883, 0.6112010759317269],
  'val_loss': 5.852380599731054,
  'val_acc': [0.2356529458402059, 0.5615623031671231, 0.7665086289246877],
  'lr': 0.01,
  'loss_weights': '2_1_0.5'},
 {'epoch': 1,
  'train_loss': 6.080811311969649,
  'train_acc': [0.19652018750791495, 0.45214269575426136, 0.6336796269410074],
  'val_loss': 5.938020700063461,
  'val_acc': [0.18563207640097693, 0.5885688639604129, 0.7551162609687219],
  'lr': 0.01,
  'loss_weights': '2_1_0.5'},
 {'epoch': 2,
  'train_loss': 6.119507001618207,
  'train_acc': [0.18565285127967765, 0.4603075832970398, 0.641798479745617],
  'val_loss': 6.0662884162022515,
  'val_acc': [0.15022957277221557, 0.5447853173200901, 0.757576013986881],
  'lr': 0.01,
  'loss_weights': '2_1_0.5'},
 {'epoch': 3,
  'train_loss': 6.167756818782138,
  'train_acc': [0.1758589619339186, 0.45036826724723233, 0.6375695606746242],
  'val_loss': 6.0802591397212105,


In [ ]:
m,h = train_model(
    DualHistogramModel(),
    train_labels,
    validation_labels,
    Constants.data_root,
    batch_size=100,
#     histogram=False,
#     lr=10,
#     upsample=False,
)
del m
h

(35354, 5)
(7786, 5)
model being saved to ../../data/models/dual_dualfacenet_h400_st600_a400_g400_ed3_std2_ad2_gd2_hist10_histd_0.1_balanced
epoch 0


In [ ]:
DualHistogramModel()

In [ ]:
import torchvision